<div style="display: flex; justify-content: center;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c4/Southwest_Airlines_logo_2014.svg/1280px-Southwest_Airlines_logo_2014.svg.png" 
         alt="Southwest Airlines Logo" width="400"/>
</div>

# Reducing Flight Delays at Southwest Airlines: A Data-Driven Strategy

_Target Audience - Senior Leadership at Southwest Airlines & Co._

## 📑 Table of Contents

1. Abstract
2. Data Descriptions  
   - Task to Tackle  
   - Data Integration  
   - Data Cleaning Strategy  
   - Checkpointing Workflow  
3. Exploratory Data Analysis (EDA)  
4. Feature Engineering  
5. Modeling Pipeline  
   - Dataset Splitting Strategy  
   - Feature Selection Strategy  
   - Final Feature Set for Modeling  
   - Feature Transformation & Vectorization  
   - Model Training & Regularization  
6. Pipeline Architecture  
7. Data Leakage Analysis  
8. Experiments  
   - Baseline Experiment  
   - Subset Analysis  
   - XGBoost Experiment  
   - Neural Network Experiment  
9. Results and Discussion  
10. Conclusion

## **Abstract**

Between 2015 and 2024, Southwest Airlines (WN) operated 14.94 million flights, of which over 3.09 million were delayed—resulting in a 20.7% delay rate and more than $2.85 billion in operational costs. To address this challenge, we developed a machine learning–powered delay prediction system designed to proactively identify flights at high risk of delay. With the potential to reduce delays by just 5%, the model could save the airline over $11 million annually and more than $140 million over a multi-year horizon. The project was executed in three phases: Phase 1 defined the problem scope and data strategy; Phase 2 focused on developing baseline models using a one-year dataset from 2015, where logistic regression achieved a recall of 0.676 and an F1 score of 0.390; and Phase 3 scaled the solution to a nine-year dataset (2015–2024), engineering features to capture temporal, spatial, and route-specific patterns. We trained and compared four models—Logistic Regression, XGBoost, Multilayer Perceptron (MLP), and Bidirectional LSTM (BiLSTM)—using rolling time-series cross-validation. Focusing on F2 score as the primary metric to prioritize recall, the best-performing model was the MLP, achieving an F2 of 0.746, recall of 0.752, and precision of 0.724 on the 2024 test set. This result demonstrates the feasibility and value of a production-ready predictive system that can integrate into Southwest’s operational workflows and drive measurable cost savings through early intervention.

</br><div style="text-align:center">
  <img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/WN_flight_delays_2015_2024.png" width="600"/>
</div>


## **Data Descriptions**
Our project uses historical, public data from the **Bureau of Transportation and Statistics** and **NOAA Weather** archived from 2015-2024, intergrating flight records, weather conditions, and airport metadata. These include:
- Flight Records from the **Bureau of Transportation and Statistics**, providing detailed operational data departures, arrivals, scheduled times, carriers, origin/destination airports, and delays status for over 14 million Southwest Airlines flights.
- Historical Weather data from **NOAA Weather Archives**, linking airports bases on geographic identifiers (latitude, longitude) and time departures. Key variable include temperature, humidity, wind speed, sky conditions, and visibility.
- Airport metadata, capturing airport codes, location identifers (latitude, longitude), and airport classifiers (regional, major hub).
- Derive time-features, allowing our model to identify seasonlity patterns in our dataset for our prediction task. 

These datasets were joined using time and spatial proximity logic. In total, the full dataset contained approx **91** million rows and **262** columns. For this phase, we filtered the data to include only flights operated by **Southwest Airlines** which boiled down the dataset size to **14** million rows and **44** columns.
Given the large dataset size, we leverage Databricks and PySpark to handle big data processing effectively.
Detail descriptions about the final features selected for modelling as well as new engineered features are mentioned in the EDA section.

---

### Task to Tackle
The task for this project is to assess the feasibility of a predictive system that can classify a flight likely to be delayed using a selected feature set. Our target variable(`DEP_DEL15`) represents a binary variable (1: delayed, 0: not delayed) for flights that depart more than 15 minutes past the scheduled departure time.

---

### Data Integration
We created our own 2015-2021 dataset by joining raw parquet files (`parquet_airlines_data`, `parquet_weather_data`, `stations_with_neighbors`), and a csv file (`airport_code_csv`) containing airport codes with location identifiers. 

 <div style="display: flex; justify-content: center;">
    <img src="https://raw.githubusercontent.com/sacayo/w261-phase2-finalproject/refs/heads/main/plots/Join_diagram.png" 
         alt="Join diagram" width="1000"/>
</div>

First, we determined the closest weather station to each airport using the `airport_codes` table and the `stations_with_neighbors` table. From the `airport_codes` table, we took the IATA Code, airport name, airport type, and converted the coordinates column into longitude and latitude. From the `stations_with_neighbors` table, we took the distinct rows containing neighbor ID, neighbor name, and neighboer latitiude and longitude coordinates. We then performed a cross join on the resulting tables and used the Haversine formula to calculate the distance between every combination of airport and weather station. 

$$d = 2 * R * asin(sqrt(sin²((φ2 - φ1)/2) + cos(φ1) * cos(φ2) * sin²((λ2 - λ1)/2)))$$

We used a window function to partition the resulting table by airport IATA code and ranked the stations by distance, filtering for the top rank for each airport. We left joined this table to the flights table at the origin and destination airports, obtaining the origin and destination latitude and longitude coordinates for the airport and the nearest station as well as the distance between the two. 

Subsequently, we manually created a table to map every state to its respective time zone and joined it to the flights table. We also generated a UTC timestamp of the scheduled departure time from the flight date and time columns and the time zone of the origin airport's state. This ensured that the UTC conversion would automatically account for daylight savings time. From this information, we calculated the UTC timestamps four hours and two hours before scheduled departure time.

Special consideration was given to address certain challenges. The flights data from 2015 to 2019 had duplicate records and required deduplication. Additionally, the `FL_DATE` column was formatted as "YYYY-MM-DD" from 2015 to 2019 and "YYYY-MM-DDTHH:MM:SS" from 2020 to 2021, so we truncated the string to the first 10 characters for consistency.

We performed the last part of the join directly in pyspark instead of SQL to take advantage of optimizations. This was a left join to bring in weather data where the origin station ID in the flights data matched the station ID in the weather data and the weather sample time was in between the four hours and the two hours before the scheduled departure time. To optimize the range join, we truncated the timestamps to full hours and first matched the hourly buckets before checking the precise time range. This dramatically reduced the number of comparisons required.

Overall the join was extremely efficient. Using 6 workers, the 1 year data took 7 minutes to join and the full data (2015-2021) took 19 minutes to join. The total file size was 6.6 GB.

During the join, we created time-based features to determine the previous flight of the same aircraft, which would prove to be useful in the final model. We used window functions to find the previous flight information ordered by the scheduled flight date and time in UTC partitioned by the tail number. These features included:
- Whether the previous flight leg was cancelled
- The origin of the previous flight leg
- The difference in numutes between the arrival time of the previous leg and the scheduled departure time of the current leg
- The number of minutes the previous flight leg's departure was delayed, and whether the delay exceeded 15 minutes
- A triplet string denoting the airport codes of the origin of the previous leg, the origin of the current leg, and the destination of the current leg

We successfully incorporated more recent flights and weather data between 2022 and 2024, directly downloading the files from the source websites and uploading them to the Databricks FileStore (DBFS). The flights data was downloaded by month, for a total of 36 csv files, which were uploaded to DBFS through the browser interface. Here, the `FL_DATE` column was formatted as "M/d/yyyy h:mm:ss a" which needed to be converted to the format consistent with the original data. The csv files were read into PySpark and then checkpointed as a parquet file.

On the other hand, the weather data was downloaded by year, for a total of 3 .tar.gz files. These files had to be uploaded to DBFS through the command line interface. We subsequently copied the compressed files from DBFS to the local driver, extracted them on the local driver, and then copied the extracted files back to DBFS. Here, we renamed the following columns to be consistent with the original data:
-     'MonthlyAverageWindSpeed': 'AWND',
-     'MonthlyHeatingDegreeDays': 'HTDD',
-     'HeatingDegreeDaysSeasonToDate': 'HDSD',
-     'MonthlyNumberDaysWithSnowfall': 'DSNW',
-     'CoolingDegreeDaysSeasonToDate': 'CDSD',
-     'MonthlyCoolingDegreeDays': 'CLDD'

We could not use the exact same join process as the original dataset because the new weather data used the Standard Hydrometerological Exchange Format (SHEF) to identify the weather stations, whereas the original dataset and the `stations_with_neighbors` table used the USAF-WBAN combination to identify the weather stations. Therefore, we created our own `stations_with_neighbors` table by finding the distinct combination of stations and their latitude and longitude coordinates which was used to compute Haversine distances. The original joined dataset and the new joined dataset were unioned by name to generate the full dataset from 2015 to 2024, which is saved in `dbfs:/student-groups/Group_04_04/df_joined_2015_2024.parquet`. 

---

### Data Cleaning Strategy

The table below summarizes the cleaning logic applied to build a usable dataset for analysis and modeling:

| **Category**             | **Columns Affected**                                                                                                           | **Missing Value Handling**                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|--------------------------|--------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Critical Columns**     | `FL_DATE`, `OP_CARRIER`, `ORIGIN`, `DEST`, `CRS_DEP_TIME`, `CRS_ARR_TIME`, `TAIL_NUM`, `DEP_DEL15`                            | - **If null:** Row is dropped.<br>- **Special for cancelled flights:** If `DEP_DEL15` is null and `CANCELLED` is 1, set `DEP_DEL15 = 1` (considered delayed) before dropping nulls.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| **Flight Schedule**      | `CRS_ELAPSED_TIME`, `DEP_TIME_BLK`, `ARR_TIME_BLK`                                                                            | - If null, derived from `CRS_DEP_TIME` and `CRS_ARR_TIME` using time difference calculations and block formatting.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| **Temporal Features**    | `YEAR`, `QUARTER`, `MONTH`, `DAY_OF_WEEK`, `DAY_OF_MONTH`                                                                     | - If null, extracted from `FL_DATE`.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| **Weather Columns**      | `HourlyWindDirection`, `HourlyAltimeterSetting`, `HourlySkyConditions`, `HourlyVisibility`, `HourlyDewPointTemperature`,<br>`HourlyWindSpeed`, `HourlyDryBulbTemperature`, `HourlyPrecipitation`, `HourlyRelativeHumidity`, `HourlySeaLevelPressure`, `HourlyStationPressure`, `HourlyWetBulbTemperature` | - Numeric columns: Imputed in order using:<br> 1. Backward-looking rolling average (past 10 rows, same origin/date)<br> 2. Historical average (past 20 rows, same origin)<br> 3. Origin/month median<br> 4. Global median<br> 5. Domain-specific default if all else fails (e.g., 0.0 for temperature, 1013.25 for pressure, etc.)<br>- `HourlySkyConditions`: Imputed by mode (most frequent) in backward window, then by month, then global mode, then default "CLR".                                                                                                                    |
| **Timestamp Columns**    | `sched_depart_date_time_UTC`, `four_hours_prior_depart_UTC`, `two_hours_prior_depart_UTC`                                     | - If null, derived from `FL_DATE` and `CRS_DEP_TIME` using timestamp arithmetic (e.g., subtracting hours from scheduled departure).                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| **Miscellaneous**        | `REM`, `WindEquipmentChangeDate`                                                                                              | - `REM`: If null, set to "UNKNOWN".<br>- `WindEquipmentChangeDate`: If null, impute with station-specific mode, then "UNKNOWN".                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |

---

### Checkpointing Workflow

To scale our development and analysis, we established multiple checkpoints:

1. **Raw Data Ingestion**  
   Load all flights and weather data for 2015 from source Parquet and CSV files.

2. **EDA & Feature Engineering**  
   Use the cleaned dataset to perform all EDA visualizations and create derived features like `holiday_season`, `DAY_HOUR`, `origin_delay_rate`, etc.

3. **Southwest Filter & Cleaning**  
   Filter for `OP_CARRIER == 'WN'` and apply the above cleaning logic.  
   ⮕ **Checkpoint #1: Cleaned Southwest Data**

4. **Train/Test/Validation Split**  
   Based on calendar years (inclusive):  
   - Training: 2015-2021
   - Validation: 2022-2023 
   - Test: 2024 
   
   ⮕ **Checkpoint #2: Final Modeling Dataset with Labels and Features**

| Dataset Partition | Years Included           | Use Case          |
|------------------|----------------------------|-------------------|
| Train            | 2015 - 2021                 | Model fitting and Hyperparameter Turning with CV     |
| Validation       | 2022 - 2023                 | Model Evaluation and Best Model Selection |
| Test             | 2024                        | Final evaluation  |

This structure allows us to simulate a production pipeline where models are trained on historical data and evaluated on unseen future years.


## Exploratory Data Analysis (EDA)

In Phase 2, we conducted an in-depth EDA on the 2015 Southwest Airlines data to clarify the challenges in predicting flight delays and to guide our feature engineering efforts. Here, we revisit the three most important findings—supported by the original charts—to set the stage for the next phase.

---

### **1. Class Imbalance: Most Flights Are On-Time**

Our initial analysis revealed a significant class imbalance: **about 77% of flights were on-time, while only 21% were delayed**. This imbalance poses a challenge for predictive modeling, as classifiers may be biased toward the majority class. For this phase, cancelled flights were dropped, but we note that in future experiments, treating cancellations as delays (since a cancellation is effectively a very long delay) may be more realistic.

<img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/flight_status_pie_chart.png" width="400"/>

---

### **2. Temporal Patterns: When Delays Occur**

The timing of a flight is strongly related to its likelihood of delay. Our analysis showed:

- **Delays peak in the late afternoon and evening (5–11 PM)**, likely due to cumulative effects throughout the day.
- **Summer (June–July)** and **holiday months (December–February)** have higher delay rates.
- **Mondays and Thursdays** are the worst days for delays.

These patterns are clear in the heatmaps and trend charts below, and suggest that operational strategies should be time-sensitive.

<p align="center">
  <img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/Heatmap_Delay_Trends.png" width="1000"/>
</p>
<table>
  <tr>
    <td><img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/Monthly_Flight_Delay_Trends.png" width="400"/></td>
    <td><img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/Day_of_week_Flight_Delay_Trends.png" width="400"/></td>
  </tr>
</table>

---

### **3. Weak Individual Predictive Power: Correlation Analysis**

We computed the Pearson correlation matrix for all numeric features and found that:

- **No single feature has strong predictive power for delays.**
- The highest (but still modest) correlation is with `CRS_DEP_TIME` (scheduled departure time).
- Weather features, distance, and even airport congestion show only weak linear relationships with delay probability.

This underscores that delays are multifactorial and not easily predicted by any one variable—highlighting the need for richer, more sophisticated feature engineering.

<img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/correlation_heatmap.png" width="800"/>

---

### **Summary and Motivation for Feature Engineering**

In summary, our EDA showed that (1) the dataset is imbalanced, (2) delay risk is closely tied to flight timing, and (3) none of the original features alone are strongly predictive. These findings motivated the next phase of our project: designing and engineering new features to improve predictive power, as described in the following section.

## Feature Engineering

Our feature engineering strategy was designed to capture the multifaceted nature of flight delays. We approached this by creating distinct feature profiles that address different aspects of the aviation ecosystem. This led us to create four high-level feature profiles:

* Airport Profile
* Time-Based Profile
* Weather-Based Profile
* Southwest Airlines Profile

Additionally, we incorporated graph-based features to capture network effects.

### Airport Profile Features

The Airport Profile features capture the operational characteristics of airports and routes that can influence flight delays. These features reflect the infrastructure constraints, historical performance, and traffic patterns that affect an airport's ability to handle flights efficiently.

For example, the **Origin Airport Daily Operations** feature captures the daily volume of flights at an airport, which directly correlates with congestion levels. Similarly, the **Route Traffic Volume** provides insights into how busy specific flight paths are, which can indicate potential bottlenecks in the system.

We also incorporated historical delay patterns through features like **Origin Airport 1-Year Delay Rate**, which helps identify chronically problematic airports. This historical perspective is crucial for predicting future delays as airports with persistent issues are likely to continue experiencing them.

The visualizations clearly demonstrate the relationship between daily airport operations and Southwest's relative performance, highlighting that busier airports tend to experience wider variability in performance, likely due to congestion and operational complexities. Additionally, the Southwest Market Share distribution illustrates that Southwest maintains varied but significant market presence across airport types.

| Feature | Description | Null Handling | Temporal Integrity |
|---------|-------------|---------------|---------------------|
| **Origin Airport Daily Operations** | Total number of flights departing from each origin airport on a given day | None needed (always populated) | Current day only |
| **Origin Airport 30-Day Rolling Volume** | Sum of flights from the origin airport over the past 30 days | 0 for first 30 days (no history available) | Growing window until 30 days history |
| **Origin Airport 1-Year Delay Rate** | Annual delay percentage at origin airport | Global fallback (15%) for first year (2015) rows | Expanding window using all prior data |
| **Route** | The origin and destination of the flight | Not needed | Concat the origin and destination in a single window |
| **Route Traffic Volume** | Number of flights between specific origin-destination pairs over the past year | 0 for new routes or first year (2015) rows | Expanding window using all prior data |
| **Southwest Market Share** | Percentage of flights operated by Southwest at each origin airport over the past year | 0 when no data available for Southwest flights | Rolling 365-day window |
| **Southwest Origin 30-Day Delay Rate** | Recent Southwest delay performance at origin airport (past 30 days) | Global fallback (15%) for missing data in the previous 30 days | Growing window until 30 days history |
| **Southwest Route Historical Performance** | Southwest's historical delay rate on specific routes over the past year | Global fallback (15%) for missing route data or first year rows | Expanding window using all prior data |
| **Southwest Relative Performance Index** | How Southwest compares to other airlines at the same airport (delay rate ratio) | Default value of 1.0 when no data available or division by zero occurs | Ratio with epsilon smoothing to prevent division by zero |

<img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/airport_profile_1.png" width="1000"/>

### Time-Based Profile Features

The Time-Based Profile features capture temporal patterns that influence flight delays. These features account for cyclical patterns in air travel demand, such as time of day, day of week, and seasonal variations.

We created features like **time_bucket** to capture the 15-minute departure intervals, which helps identify peak congestion periods within each hour. The **time_of_day_category** feature groups flights into morning, midday, evening, and night categories, recognizing that different times of day have distinct operational characteristics.

The **holiday_season** indicator identifies peak travel periods when airports and airlines operate under heightened stress. We also incorporated dynamic features like **prior_day_delay_rate** and **same_day_prior_delay_percentage** to capture the cascading effects of delays, where problems earlier in the day or from the previous day can propagate through the system.

The provided visualizations effectively demonstrate a clear linear relationship between prior day's delay rates and same-day delay percentages, confirming that delays frequently cascade from one day to the next. Additionally, delays significantly vary throughout the day, with flights during evenings and nights experiencing notably higher delay rates compared to mornings.

| Feature | Description | Null Handling | Temporal Integrity |
|---------|-------------|---------------|---------------------|
| **time_bucket** | 15-minute departure intervals | Derived from CRS_DEP_TIME (always populated) | Current flight only |
| **dep_hour** | Hour of day for scheduled departure | None needed | Current flight only |
| **time_of_day_category** | Morning/Midday/Evening/Night | Categorical fallback to "night" | Current flight only |
| **is_weekend** | Weekend flight indicator | None needed | Current flight only |
| **holiday_season** | Peak travel period indicator | None needed | Current flight only |
| **prior_day_delay_rate** | Previous day's delay rate at origin airport | 3-level fallback: prior day → airport avg → 15% global fallback | Strict date ordering |
| **same_day_prior_delay_percentage** | Percentage of flights delayed earlier in the day at the same airport | Additive smoothing (prevents 0/0) and nulls default to 0% delay rate | Same-day ordering |
| **time_based_congestion_ratio** | Current vs historical congestion ratio for the same time bucket (hour + 15-min interval) on the same day of the week at the same airport | 3-level fallback: historical average → airport avg → default capacity (10 flights) | 365-day lookback excluding current day |

<img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/time_based_features_1.png" width="1000"/>

### Weather-Based Profile Features

Weather conditions are among the most significant external factors affecting flight operations. Our Weather-Based Profile features capture both extreme weather events and subtle weather patterns that can impact flights.

Rather than simply using raw weather measurements, we engineered features that identify extreme conditions relative to each airport's historical norms. For example, extreme_precipitation flags precipitation levels that exceed the 95th percentile for a specific airport, recognizing that what constitutes "heavy rain" varies by location.

We also created a composite extreme_weather_score that combines multiple weather factors weighted by their historical impact on delays. This provides a single, comprehensive measure of weather risk. Additionally, features like rapid_weather_change capture sudden shifts in conditions that can disrupt operations even when absolute values aren't extreme.

| Feature | Description | Calculation Method | Null Handling |
|---------|-------------|--------------------|---------------|
| **extreme_precipitation** | Flag for heavy precipitation | 95th percentile of historical precipitation data | 0 if missing |
| **extreme_wind** | Flag for high wind conditions | 95th percentile of historical wind speed data | 0 if missing |
| **extreme_temperature** | Flag for extreme temperatures | 5th/95th percentiles of historical temperature data | 0 if missing |
| **low_visibility** | Flag for poor visibility | 5th percentile of historical visibility data | 0 if missing |
| **extreme_weather_score** | Weighted weather risk score | Weighted sum of extreme conditions based on their historical delay impact | Scaled to [-1,1] |
| **heat_index** | Perceived temperature | NOAA heat index formula for T ≥ 80°F and RH ≥ 40% | Raw temp otherwise |
| **rapid_weather_change** | Significant weather shifts | Z-score > 3 in temp/wind over 24h window | 0 if missing data |
| **temp_anomaly_z** | Temperature deviation | Z-score vs. airport-month historical average | 0 if no history |
| **precip_anomaly_z** | Precipitation deviation | Z-score vs. airport-month historical average | 0 if no history |

### Southwest Airlines Profile Features

Since our analysis focuses specifically on Southwest Airlines, we created a dedicated profile of features that capture Southwest's operational characteristics and performance patterns.

These features include sw_time_of_day_delay_rate, which captures Southwest's historical performance during specific time periods at each origin airport. The sw_aircraft_delay_rate provides insights into the performance of individual aircraft in Southwest's fleet, using a hierarchical approach that falls back to route-level and then global averages when specific aircraft data is limited.

We also engineered features like sw_origin_hub to identify Southwest's operational hubs dynamically based on flight volume, and sw_schedule_buffer_ratio to capture how Southwest's scheduling practices compare to historical norms. The sw_route_importance feature helps identify routes that are particularly significant to Southwest's network, combining both frequency and distance considerations.

Southwest Airlines profile visualizations clearly indicate consistent performance across different airport types, with large airports showing more variability due to operational complexity and congestion.


| Feature | Description | Calculation Method | Null Handling |
|---------|-------------|--------------------|---------------|
| **sw_time_of_day_delay_rate** | Southwest's delay rate by origin and time bucket | Expanding window average with origin/global fallbacks | Uses origin average → global median |
| **sw_day_of_week_delay_rate** | Bayesian-smoothed delay rate by route and weekday | (Delays + 3*global_p30)/(Flights + 3) | Built-in smoothing prevents nulls |
| **sw_aircraft_delay_rate** | Aircraft performance metric | Hierarchical: aircraft → route → global median | Always populated |
| **sw_origin_hub** | Dynamic hub identification | Top 15th percentile of Southwest flight volume | 0/1 encoding |
| **sw_schedule_buffer_ratio** | Schedule padding ratio | Current vs 1-year historical average | Defaults to 1.0 |
| **sw_origin_time_perf** | Hybrid airport/time performance | Time bucket → time category → global fallback | Hierarchical coalesce |
| **sw_route_importance** | Normalized route significance | (Flight count + distance) normalized | Always 0-2 range |

<img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/sw_specific_features_1.png" width="1000"/>

### Graph-Based Features

To capture the network effects within Southwest's route system, we implemented several graph-based algorithms using PySpark's GraphFrames module. These features provide insights into the structural importance of airports within Southwest's network and how delays might propagate through connected airports.

In our graph representation, airports serve as vertices while flights between them form directed edges. This allows us to analyze the flow of traffic and influence throughout the network.

The PageRank algorithm identifies influential airports based on their connections to other important airports. For example, Hartsfield–Jackson Atlanta International Airport (ATL) emerged as having the highest lagged PageRank destination value for 2016, reflecting its role as a major connection hub for Southwest flights. PageRank values were calculated annually and lagged by year to capture seasonal patterns while maintaining temporal integrity.

We complemented PageRank with InDegree and OutDegree metrics, which measure the volume of incoming and outgoing flights at each airport. These metrics were calculated quarterly to capture seasonal variations in traffic patterns. Airports with high InDegree values represent major destination hubs, while those with high OutDegree values serve as significant origin points in Southwest's network.

Visualizations of graph-based features show that airports with higher PageRank or OutDegree tend to have higher average delay rates, highlighting the operational risks associated with central or highly connected hubs in the network.

Together, these graph-based features provide a multi-dimensional view of airport connectivity and importance within Southwest's operational network, helping our model understand how delays might cascade through the system.

| Graph Feature Category         | Description             | **Calculation Method** | **Lag method** |
|--------------------------|----------------------------|-----------------------------------------------------------------------------|--------------------------------------------------------------------------|
| `PageRank`    | Measure of influence of high-traffic airports based on flight connection       | Distinct airport ID as Vertices and flight routes as Edges (src: origin airport ids, dst: destination airport ids) |  Year |
|  `InDegree`   | Measure of high-traffic airport arrival patterns | Count of incoming connections from an airport | Quarter |
|   `OutDegree`   | Measure of high-traffic airport departure patterns  | Count of outgoing connections from an airport | Quarter |

<img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/graph_based_features_1.png" width="1200"/>


## Modeling Pipeline

With the dataset cleaned, features engineered, and delays dissected through exploratory analysis, we now move to the modeling phase. The task is to build a binary classifier that predicts if a Southwest Airlines flight will be delayed at departure by 15 minutes or more (`DEP_DEL15` = 1 for delayed, 0 for on-time).

---

### 1. Dataset Splitting Strategy

To simulate a real-world pipeline where training happens on historical data and evaluation on future quarters, we split our 2015 dataset into Train, Validation, and Test sets based on calendar years:

| Dataset Partition | Years Included         | Use Case              |
|-------------------|--------------------------|------------------------|
| Train             | 2015 - 2021              | Model fitting         |
| Validation        | 2022 - 2023              | Hyperparameter tuning |
| Test              | 2024                     | Final evaluation      |

---

### 2. Feature Selection Strategy

The dataset initially had **262 columns**. We adopted a structured feature selection pipeline involving multiple filtering and evaluation stages:

1. **Null-based filtering**: Dropped columns with more than 50% missing values → 94 columns retained.  
2. **Manual domain-driven pruning**: Removed 15 leakage-prone columns.  
3. **Initial subset**: Resulted in 79 columns.  
4. **Feature Engineering**: Added robust temporal, weather, airline-specific and graph features → 117 columns.  
5. **Manual inspection**: Pruned down to 52 columns.  
6. **Pearson correlation (numerical)**: Reduced to 49.  
7. **Spearman correlation (categorical/ordinal)**: Final selected features → **44 columns**.

Feature selection methods included **Pearson correlation for numerical variables** and **Spearman correlation for categorical or ordinal features**, balancing statistical relevance with domain knowledge. 

<div style="display: flex; justify-content: space-between; gap: 20px;">

  <img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/perason.png" width="49%"/>

  <img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/spearman.png" width="49%"/>

</div>

---

### 3. Final Feature Set for Modeling

Below are the final 44 features used for model training, grouped by category:

#### 📆 Date/Time Features
| Feature Name        | Description                                  |
|---------------------|----------------------------------------------|
| `DAY_OF_MONTH`      | Day of the month the flight departs          |
| `DAY_OF_WEEK`       | Day of the week the flight departs           |
| `YEAR`              | Flight year                                  |
| `MONTH`             | Flight month                                 |
| `year_quarter`      | Combined year and quarter                    |
| `quarter_month`     | Month within the quarter                     |

#### 🛫 Airport Profile Features
| Feature Name            | Description                                              |
|-------------------------|----------------------------------------------------------|
| `origin_type`           | Size category of origin airport (small/medium/large)     |
| `dest_type`             | Size category of destination airport                     |
| `daily_operations`      | Total flights departing origin on the day                |
| `rolling_30day_volume`  | 30-day rolling flight count at origin                    |
| `origin_1yr_delay_rate` | 1-year average delay rate at origin                      |

#### 🛬 Route Information
| Feature Name         | Description                                                            |
|----------------------|------------------------------------------------------------------------|
| `TRIPLET`            | Concatenated route of previous and current legs                        |
| `route`              | Origin-destination pair identifier                                     |
| `route_1yr_volume`   | Total flights on route over the past year                              |
| `sw_market_share`    | Southwest's share of total origin traffic                              |
| `sw_30d_delay`       | Southwest 30-day delay rate at origin                                  |
| `sw_route_delay`     | Southwest average delay on this route                                  |
| `sw_rel_perf`        | Southwest delay rate vs airport average                                |
| `sw_route_importance`| Normalized score of route importance to Southwest                      |

#### 🕒 Time-Based Features
| Feature Name                    | Description                                                     |
|--------------------------------|-----------------------------------------------------------------|
| `time_bucket`                  | 15-minute interval for scheduled departure                      |
| `time_of_day_category`         | Categorized time of day: morning/evening/etc.                   |
| `is_weekend`                   | Indicator for weekend flights                                   |
| `holiday_season`              | Indicator for peak travel month                                 |
| `prior_day_delay_rate`         | Delay rate at origin airport on previous day                    |
| `prior_flights_today`          | Number of flights earlier in the same day                       |
| `prior_delays_today`           | Count of delayed flights earlier today                          |
| `same_day_prior_delay_percentage` | Fraction of delays prior to this flight                     |
| `time_based_congestion_ratio`  | Current congestion vs historical average                        |

#### ✈️ Southwest Profile Features
| Feature Name                 | Description                                                       |
|------------------------------|-------------------------------------------------------------------|
| `sw_time_of_day_delay_rate` | Historical SW delay rate by time of day                           |
| `sw_day_of_week_delay_rate` | SW delay rate by day of week on route                             |
| `sw_aircraft_delay_rate`    | Average delay for aircraft (TAIL_NUM)                             |
| `sw_origin_hub`             | Whether origin airport is a Southwest hub                         |
| `sw_schedule_buffer_ratio` | Flight padding relative to historical average                     |
| `sw_origin_time_perf`       | SW performance by airport + time bucket                           |

#### 🌐 Lagged Graph-Based Features
| Feature Name               | Description                                            |
|----------------------------|--------------------------------------------------------|
| `pagerank_origin_lag`      | Lagged PageRank score of origin airport               |
| `pagerank_destination_lag`| Lagged PageRank score of destination airport          |
| `origin_indegree_lag`      | Lagged indegree of origin airport                    |
| `origin_outdegree_lag`     | Lagged outdegree of origin airport                   |
| `dest_indegree_lag`        | Lagged indegree of destination airport               |
| `dest_outdegree_lag`       | Lagged outdegree of destination airport              |

#### 🧩 Other Raw Inputs
| Feature Name             | Description                                      |
|--------------------------|--------------------------------------------------|
| `DEP_DEL15`              | Target variable: delayed departure (1=yes)       |
| `PREV_CANCELLED`         | Whether previous leg was cancelled               |
| `MINUTES_BETWEEN_FLIGHTS`| Gap in minutes from previous flight              |
| `PREV_ARR_DEL15`         | Whether the previous leg arrived late            |

---

### 4. Feature Transformation & Vectorization

To prepare the data for modeling:

- **Categorical Features** were one-hot encoded using `OneHotEncoder` from `pyspark.ml`
- **Numerical Features** were standardized using `StandardScaler`
- All transformed columns were **assembled into a single feature vector** using `VectorAssembler`, forming the final modeling input under column `features`.

---

### 5. Model Training & Regularization

We trained three models using concepts like **regularization, grid search, and cross-validation**, employing **Optuna** for hyperparameter tuning with **time-series-aware cross-validation blocks**:

- **Logistic Regression** served as our baseline model.  
- **XGBoost** provided gradient-boosted tree performance.  
- **Neural Networks** enabled flexible non-linear modeling.

Details about model-specific experiments, compute time, and performance are described in later sections.

We evaluated the models using:
- **Precision**: How many predicted delays were actually delayed.
- **Recall**: Ability to detect actual delays (critical for minimizing false negatives).
- **F2 Score**: Emphasizes recall more than precision — crucial in our domain.

---

### 5.1 Regularization

Given the sparsity of our feature space, we attempt to apply **Lasso Regularization** for potential feature selection, a technique that penalizes large or complex coefficients in the model and encourages generalization by shrinking less useful weights toward zero.

Regularization is introduced into the logistic loss function by adding a penalty term to the objective:

#### Generalized Regularized Logistic Loss:

$$
\\mathcal{L}_{\\text{total}} = \\mathcal{L}_{\\text{data}} + \\lambda \\cdot \\mathcal{R}(w)
$$

We explored two primary forms:

#### Lasso (L1) Regularization

Promotes **sparse solutions** by shrinking some weights to zero. This is useful for implicit feature selection.

$$
\\mathcal{R}_{L1}(w) = \\sum_{j=1}^{p} |w_j|
$$

#### Elastic Net (L1+L2) Regularization

In addition to pure Lasso Regularization, we also try Elastic Net Regularization, which is a combination of Lasso and Ridge Regularization.

$$
\\mathcal{R}_{L1+L2}(w) = \\sum_{j=1}^{p} |w_j| + \\sum_{j=1}^{p} w_j^2
$$

---

### 5.2 Model Selection Strategy

**Recall and F2 Score** are the primary evaluation metrics.

- Low recall implies high false negatives — i.e., delayed flights predicted as on-time, which is risky for airlines.  
- F2 Score balances precision and recall but **emphasizes recall**, making it well-suited for delay prediction.

We also monitored **precision** to ensure false positives (over-predicting delays) are reasonably controlled.

### Pipeline Architecture

Below is the end-to-end pipeline diagram summarizing the complete flow:

<div style="display: flex; justify-content: center;">
    <img src="https://raw.githubusercontent.com/sacayo/w261-phase2-finalproject/refs/heads/main/plots/Phase3_model_Pipeline.png" 
         alt="ML Pipelines diagram" width="1000"/>
</div>

This architecture allowed us to reuse checkpoints, track transformations efficiently, and iterate rapidly across modeling experiments.


## Data Leakage Analysis

### What is Data Leakage?

Data leakage occurs when information from outside the training dataset is used to create a model, allowing it to make predictions with artificially high accuracy. This happens when data that would not be available at prediction time is inadvertently included during training.

A hypothetical example in the context of flight delay prediction: If we included the actual departure delay time (`DEP_DELAY`) as a feature to predict whether a flight was delayed by 15+ minutes (`DEP_DEL15`), this would constitute severe leakage since the target variable is directly derived from the feature. The model would achieve near-perfect accuracy but would be useless in practice since the actual departure delay wouldn't be known when making predictions.

### Leakage Analysis of Current Feature Set

#### Temporal Features and Leakage Prevention

Our feature engineering demonstrates awareness of temporal dependencies:

- **Proper Lagging of Time-Based Features**: Features like `prior_day_delay_rate`, `sw_30d_delay`, and `origin_1yr_delay_rate` use historical data rather than future data.
- **Graph Features with Appropriate Temporal Boundaries**: The `PageRank`, `InDegree`, and `OutDegree` features are properly lagged, ensuring that future information doesn't leak into the training process.
- **Same-Day Features with Careful Implementation**: The `same_day_prior_delay_percentage` only considers flights that have already departed, maintaining the temporal integrity of our model.

#### Potential Leakage Concerns

While our feature engineering is generally sound, there are a few areas that warrant careful consideration:

- **Graph Feature Imputation**: As noted, the imputation strategy for 2015 `PageRank` values and 2015-Q1 `InDegree`/`OutDegree` values involves using the same period's data due to the lack of prior data. This creates a minor risk of leakage, but our mitigation strategy is appropriate:
  - Confining this imputation to the training set (2015-2017)
  - Keeping validation (2018) and test (2019) sets completely separate
  - Using this as a temporary solution rather than a predictive enhancement
- **Flight-Specific Features**: Features like `PREV_CANCELLED`, `MINUTES_BETWEEN_FLIGHTS`, and `PREV_ARR_DEL15` need careful implementation to ensure they only use information from previous flights, not concurrent or future ones.
- **Time-Based Aggregations**: Features like `time_based_congestion_ratio` must be calculated using only historical data at each point in time, which appears to be the case in our implementation.

### Cardinal Sins of ML and Our Pipeline

- **Training-Test Contamination**: Our pipeline properly separates data chronologically (2015-2017 for training, 2018 for validation, 2019 for testing), which prevents this sin.
- **Target Leakage**: Our features appear to be constructed without using information that would not be available at prediction time. None of our features directly incorporate the target variable (`DEP_DEL15`) or its derivatives.
- **Data Snooping**: Our feature engineering process seems to be guided by domain knowledge rather than repeatedly testing against the validation set, which helps avoid this sin.
- **Temporal Leakage**: Our time series cross-validation approach respects the temporal nature of the data, preventing models from being trained on future data.
- **Feature Selection Bias**: While not explicitly mentioned, our comprehensive feature set suggests a thoughtful selection process rather than automated selection that might introduce bias.

### How Our Pipeline Prevents Leakage

Our pipeline incorporates several best practices that effectively prevent data leakage:

- **Chronological Data Splitting**: By strictly separating training (2015-2017), validation (2018), and test (2019) data chronologically, we ensure that models are evaluated on truly unseen future data.
- **Temporal Feature Engineering**: Features like `rolling_30day_volume`, `prior_day_delay_rate`, and `sw_30d_delay` are constructed using only historical information available at prediction time.
- **Proper Handling of High-Cardinality Features**: Our approach to high-cardinality features like `route` and `time_bucket` avoids leakage by encoding them appropriately.
- **Cross-Validation Strategy**: Our time series cross-validation approach respects the temporal ordering of data, preventing future information from influencing the training process.
- **Graph Feature Implementation**: Despite the necessary imputation for 2015 data, our approach minimizes leakage risk by containing it within the training set and ensuring validation and test sets remain unaffected.

### Conclusion

Our flight delay prediction pipeline demonstrates understanding of data leakage concerns and implements appropriate safeguards. The feature engineering process respects temporal boundaries, the data splitting strategy maintains chronological integrity, and the few instances where compromises were necessary (like the 2015 graph feature imputation) are handled with appropriate caution.

The pipeline does not appear to violate any cardinal sins of machine learning and incorporates best practices for preventing data leakage in time series forecasting. The comprehensive feature set, spanning airport profiles, time-based metrics, and graph-based network analysis, provides a rich foundation for prediction without introducing artificial advantages through leaked information.

## Experiments
All models are trained with the same set of features listed above. Each model varied in Databricks Cluster (CPUs vs GPUs) and training speed. Each model may have different hyperparameter tuning approach (Grid Search vs Optuna Bayesian Search). The cross validation streatgy is 4-fold rolling time series cross validation to avoid any leakage. For each fold, we split the time-ordered training data by threshold `[0.2, 0.4, 0.6, 0.8]` respectively into training data for CV and validation data for CV; for example, for the third fold, the first 60% of the training data, ordered from the earliest time to the latest, is used for CV training, and the rest 40% is used for validation.

#### 1. Baseline Experiment: Logistic Regression

As a baseline model, we trained a **Logistic Regression (LR)** classifier using Spark ML. The first version of this model was trained **without regularization**, but with **class-balanced sample weights**, computed as:

$$
\mathcal{SW}_{\text{delayed}} = \frac{\text{totalCounts}}{2 \times \text{totalDelay}}, \quad 
\mathcal{SW}_{\text{onTime}} = \frac{\text{totalCounts}}{2 \times \text{totalOnTime}}
$$

The cluster configuration and model training time is as follows:
- **Driver**: `m6gd.8xlarge` (128 GB RAM, 32 Cores)
- **Workers (10)**: `m6g.2xlarge` (32 GB RAM, 8 Cores each)
- **Runtime**: Databricks Runtime `15.4 LTS ML` with Spark 3.5.0
- **Model Training Time (including Hyperparameters Tuning with Cross Validation)**: 1.1 hour

As detailed in Phase 2, we have performed hyperparameter tuning for a logistic regression model with Lasso Regularization, under rolling time series cross validation. However, no better performance seen. Therefore, the unregularized baseline is selected as our final baseline model, serving as a strong benchmark for comparison with tree based and deep learning approaches. Predictions were generated across train, validation, and test sets using this unregularized baseline. The results below show the model's precision, recall, F2 score, and confusion matrices.

<br/><img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/lr_results.png" width="1000"/>

#### 2. Subset analysis
As part of our baseline experimentation process, we conducted a subset analysis to evaluate the performance of our Logistic Regression model. Specifically, we compared model results trained on our Southwest-only dataset against a broader dataset with all unique carriers for the 3 years — training set was for years 2015-2017. To ensure consistency, we applied the same datacleaning, feature engineering, train/test splitting, and modeling process to both datasets. The key difference between the two lies in the size and composition of the training data:

* The full carrier dataset included approximately 18.15 million training records (more than four times larger than the Southwest-only training set, which contained 4.19 million records.
* For both models, validation and test sets were derived using the same methodology. We filtered for Southwest-only before modeling.

A full table of the size of both dataset can be found below. Despite the significant increase in training data, the model trained on all carriers underperformed on key evaluation metrics.  Across the board we found the precision, recall, and F2 score were lower when compared to the Southwest-only baseline. This suggests that carrier-specific patterns may be critical for predictive performance, and that broader datasets may introduce noise that diminishes model accuracy when applied to Southwest operations.

| Datasets | Training Size | Validation Size | Test Size |
|---------|--------------|--------------|--------------|
| **Southwest-only** | 4,192,913 | 1,476,345 | 1,471,235 |  
| **Full Carrier Training Set** |18,153,200  |1,476,345 | 1,471,235 |  



#### 3. XGBoost Experiment

A XGBoost with default hyperparameters (n_estimators=100, max_depth=6, learning_rate=0.3) is fitted, achieving a validation precision of 0.872, a recall of 0.696, a f2 of 0.725. Then, we apply hyperparameter tuning with rolling time series cross validation for a better set of n_estimators, max_depth, and learning_rate. Unlike the logistic regression model, we use Optuna, a hyperparameter optimization framework powered by Bayesian Search, for a more efficient XGBoost model hyperparameters search.

The cluster configuration and model training time is as follows:
- **Driver**: `m6gd.8xlarge` (128 GB RAM, 32 Cores)
- **Workers (10)**: `m6g.2xlarge` (32 GB RAM, 8 Cores each)
- **Runtime**: Databricks Runtime `15.4 LTS ML` with Spark 3.5.0
- **Model Training Time (including Hyperparameters Tuning with Cross Validation)**: 2.47 hours

Optune yields a set of hyperparameters:
- **n_estimators**: 115
- **max_depth**: 9
- **learning_rate**: 0.27122395210228795

The following is the evaluation metrics and confusion matrix:
<br/><img src="https://raw.githubusercontent.com/fan005mids/DS261FinalProj/refs/heads/main/Phase3%20XGBoost%20Model%20Eval.png"/>

#### 4. Neural Network Experiment

This section describes three key experiments conducted to train and tune a feed-forward neural network (NN) model for binary classification of flight delays. All experiments were run on a GPU-enabled Databricks cluster with the following configuration:

- **Driver**: `g4dn.8xlarge` (128 GB RAM, 32 vCPUs)
- **Workers (4)**: `g4dn.xlarge` (64 GB RAM, 16 vCPUs each)
- **Runtime**: Databricks Runtime `15.4 LTS ML` with Spark 3.5.0

---

##### **Experiment 1: Manual Grid Search without Cross-Validation**

In this experiment, a basic grid search was performed manually over a small set of predefined hyperparameter configurations. Each model was trained using early stopping on the validation set (patience = 5). The evaluation was done across **train**, **validation**, and **test** splits.

- **Date Range**: 2015–2021  
  - **Train**: 2015–2019  
  - **Validation**: 2020  
  - **Test**: 2021  
- **Training Time**: ~27 minutes

**Grid Search Space:**

```python
param_grid = [
    {'hidden_layers': [128], 'batch_size': 256, 
     'learning_rate': 0.0005, 'dropout_rate': 0.2},
    {'hidden_layers': [128, 64], 'batch_size': 512,
     'learning_rate': 0.0003, 'dropout_rate': 0.3},
    {'hidden_layers': [256, 128, 64], 'batch_size': 1024,
     'learning_rate': 0.0001, 'dropout_rate': 0.5},
    {'hidden_layers': [128], 'batch_size': 256,
     'learning_rate': 0.001, 'dropout_rate': 0.4}
]
```


<br/><img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/NN_table_comparision.png" width="1000"/>


<br/><img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/NN_confusion_matrix.png" width="1000"/>

From the comparison table and confusion matrices, the best-performing configuration was the deepest model with three hidden layers `[256, 128, 64]`, a batch size of 1024, a learning rate of 0.0001, and a relatively high dropout rate of 0.5. This configuration achieved the **highest F2 score on the validation set (0.7135)** and a **strong F2 score on the test set (0.6947)**, indicating a well-generalized model with a good balance between precision and recall for the delayed class. The confusion matrices show consistent behavior across train, validation, and test splits, with slightly more false negatives than false positives—a common trade-off when optimizing for recall-heavy metrics like F2. Overall, this setup offered the most effective performance in the manual search and served as a strong candidate to compare against the more automated tuning process in Experiment 2.

         
##### **Experiment 2: Hyperparameter Optimization with Cross-Validation (Optuna)**

In this experiment, a more automated and systematic hyperparameter tuning was performed using Optuna with time-series cross-validation. Each trial trains a model across 4 time-based CV folds, optimizing for the F2 score on the validation splits.

This setup allows for better generalization and temporal robustness in a time-sensitive prediction task like flight delays.

- **Date Range**: 2015–2024
  - **Train**: 2015–2021
  - **Validation**: 2022–2023
  - **Test**: 2024
- **Training Time**: 
  - To find best hyperparameters: ~1 hour 30 minutes 
  - To train final model: ~45 minutes
- **Optuna Trials**: 1 trial (extendable)
- **Search Method**: Tree-structured Parzen Estimator (TPE)
- **Objective**: Maximize average F2 score across folds

**Search Space**:

- Number of hidden layers: 1–3
- Units per layer: 32–512
- Dropout: 0.1–0.5
- L2 regularization: 1e-5 – 1e-2
- Batch size: 128, 256, 512, 1024
- Learning rate: 1e-5 – 1e-2
- Batch normalization: True / False

**Best hyperparameters:**

- **n_layers**: 1
- **units_l0**: 448
- **dropout_rate**: 0.27565986839411283
- **l2_reg**: 0.0001554892275405439
- **batch_size**: 256
- **learning_rate**: 1.1258097381851819e-05
- **use_batch_norm**: False

After training with the best parameters, the model achieved the following performance:

| Split | F2 Score | Recall | Precision |
|-------|----------|--------|-----------|
| Train | 0.7488   | 0.7555 | 0.7234    |
| Val   | 0.7581   | 0.7638 | 0.7362    |
| Test  | 0.7469   | 0.7527 | 0.7243    |

<br/><img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/NN_best_model.png" width="1000"/>

The confusion matrices show strong alignment across train, validation, and test sets — indicating a stable and robust generalization performance.

##### **Experiment 3: Bidirectional LSTM (BiLSTM)**

While the first two experiments focused on **multilayer perceptrons (MLP)**, this third experiment explores a **Bidirectional Long Short-Term Memory (BiLSTM)** network to assess whether sequential modeling over feature space improves performance.

Taking inspiration from recent research ([Bisandu and Moulitsas (2023)](https://dspace.lib.cranfield.ac.uk/server/api/core/bitstreams/1b1c8945-a27d-47cf-b8db-4b6481cbb536/content)), which suggests the effectiveness of LSTMs in capturing temporal and contextual dependencies in flight delay data, we implemented a BiLSTM-based architecture.

The BiLSTM model processes the flattened feature vectors as 1-timestep sequences, allowing it to model directional relationships between features. Although this isn’t a traditional sequential dataset (like text or time-series per row), treating features as sequences can still allow LSTM to capture higher-order interactions.

- **Date Range**: 2015–2024
  - **Train**: 2015–2021
  - **Validation**: 2022–2023
  - **Test**: 2024
- **Training Time**: ~1 hour 27 minutes
- **Model Type**: Single-layer BiLSTM
- **Batch Size**: 256 (Optuna-inspired)
- **Dropout**: 0.3
- **L2 Regularization**: 0.0001
- **Learning Rate**: 1e-4

While this experiment did not undergo exhaustive hyperparameter tuning due to compute constraints, it served as a proof of concept and demonstrated encouraging results:

| Split | F2 Score | Recall | Precision |
|-------|----------|--------|-----------|
| Train | 0.7284   | 0.7196 | 0.7660    |
| Val   | 0.7210   | 0.7070 | 0.7833    |
| Test  | 0.7149   | 0.7030 | 0.7667    |

<br/><img src="https://raw.githubusercontent.com/ayushigoel9/w_261_final_project/main/BiLSTM_results.png" width="1000"/>

As shown in the confusion matrices, the BiLSTM model produced a **slightly more balanced distribution** of false positives and false negatives compared to MLP models. While precision increased on the validation and test sets, the F2 score remained in the same ballpark — indicating that BiLSTM offers **comparable performance** with a different modeling perspective.

Future work could involve expanding this architecture with:
- More LSTM layers
- Attention mechanisms
- Tuning sequence length or embedding temporal hierarchies

This experiment highlights the **potential of sequential modeling** for even non-time-sequential tabular features in flight delay prediction.

## Result and Discussions


This section interprets and compares the performance of all models explored in our experiments — including Logistic Regression (baseline), XGBoost, Feedforward Neural Network (MLP), and BiLSTM. Each model was evaluated using consistent data splits and metrics (Precision, Recall, F2), with a particular focus on **F2-score** due to its emphasis on recall for the delayed class.

All models were evaluated on the same temporal splits to ensure fair comparison:  
- **Train**: 2015–2021  
- **Validation**: 2022–2023  
- **Test**: 2024  

---

### Model Performance Comparison

| Metric    | Logistic Regression | XGBoost        | Neural Network (MLP) | BiLSTM         |
|-----------|---------------------|----------------|-----------------------|----------------|
| **Train F2**     | 0.725               | **0.752**        | 0.748                 | 0.728          |
| **Train Recall** | **0.759**           | 0.722            | 0.755                 | 0.720          |
| **Train Precision** | 0.614           | **0.897**        | 0.723                 | 0.766          |
| **Val F2**       | 0.754               | 0.733            | **0.758**             | 0.721          |
| **Val Recall**   | **0.802**           | 0.707            | 0.764                 | 0.707          |
| **Val Precision**| 0.609               | 0.863            | 0.736                 | **0.783**      |
| **Test F2**      | 0.732               | 0.744            | **0.746**             | 0.715          |
| **Test Recall**  | 0.777               | 0.730            | **0.753**             | 0.703          |
| **Test Precision**| 0.595              | 0.760            | 0.724                 | **0.767**      |

---

### Confusion Matrix: Best MLP Model on Test Set

<div style="display: flex; justify-content: center;">
    <img src="https://raw.githubusercontent.com/fan005mids/DS261FinalProj/refs/heads/main/Phase%203%20NN%20Test%20Eval.png" 
         alt="Test Data Confusion Matrix" width="700"/>
</div>

---

### Interpretation and Model Insights

- **Logistic Regression**  
  - On the **validation set**, the model achieved the highest **recall** (0.802) among all models, but with low **precision** (0.609), resulting in many false positives.
  - On the **test set**, its F2 dropped to 0.732, with **recall** at 0.777 but **precision** at just 0.595.
  - While recall-heavy, its poor precision makes it less suitable for operational use, where false alarms can be costly.

- **XGBoost**  
  - On the **validation set**, XGBoost had the highest **precision** (0.863), but lower **recall** (0.707), leading to a validation F2 of 0.733.
  - On the **test set**, it maintained balanced performance with **F2 = 0.744**, **precision = 0.760**, and **recall = 0.730**.
  - This pattern suggests that XGBoost prioritizes precision and is slightly prone to underpredicting delays.

- **Neural Network (MLP)**  
  - On the **validation set**, the MLP achieved the **best F2 score (0.758)** with a strong balance of **recall (0.764)** and **precision (0.736)**.
  - On the **test set**, it continued to perform consistently with **F2 = 0.746**, **recall = 0.753**, and **precision = 0.724**.
  - This consistency across splits indicates that the model generalizes well without overfitting and aligns best with our F2-optimized objective.

- **BiLSTM**  
  - On the **validation set**, BiLSTM showed the **highest precision** (0.783), even higher than XGBoost, but recall was slightly lower at 0.707, leading to **F2 = 0.721**.
  - On the **test set**, its performance was similar, with **F2 = 0.715**, **recall = 0.703**, and **precision = 0.767**.
  - This indicates that BiLSTM is more conservative, prioritizing precision and correctly identifying delays it is more confident about. While its F2 was slightly lower, the precision-focused behavior could be desirable in certain use cases (e.g., limited intervention resources).

---

### Model Selection Reasoning

We selected the **Feedforward Neural Network (MLP)** as our best model based on:
- **Highest validation and test F2 scores**
- Consistent performance across all data splits
- Balanced trade-off between recall and precision

While XGBoost was more precise, it underperformed in recall. BiLSTM offered better precision than MLP, but its recall was lower, making it less suitable for our F2-optimized goal. The baseline Logistic Regression also lacked the precision needed to avoid unnecessary operational escalations.

---

### Gap Analysis

In 2024, Southwest Airlines experienced 340,115 delays. Our model, during testing, successfully identified 256,012 of these delays. Considering that our model has a recall rate of over 75%, we are confident that it can accurately identify three-quarters of the total delay.Early identification of delayed flights is crucial. If we can prepare in advance and achieve a 5% cost reduction, we could save approximately 11.7 million dollars in 2024. A 10% cost reduction would yield savings of over 23 million dollars.We also conducted a long-term analysis spanning six years from 2015 to 2021, assuming a 10% cost reduction. This analysis projected a savings of approximately 140 million dollars. Detailed calculations are available in the appendix.

Our next step involves collaborating with all stakeholders to plan a soft launch and integrate the model into our daily operations. This will enable the corporation to start enjoying these cost savings as soon as possible.


## Conclusion


Southwest Airlines faces a significant operational challenge in predicting flight delays, which have resulted in substantial financial losses of over **$2.84 billion** between 2015 and 2024. We hypothesized that a machine learning pipeline, equipped with meticulously crafted features, could reliably forecast flight delays before departure. This would enable proactive planning and data-driven interventions to mitigate operational disruptions.

In **Phase 2**, we validated this hypothesis using a 12-month dataset from 2015. We integrated flight-level data from the Department of Transportation (DOT) and weather records from the National Oceanic and Atmospheric Administration (NOAA). We constructed an end-to-end machine learning pipeline and trained a baseline **Logistic Regression** model. Despite its simplicity, the model demonstrated promising early potential in predicting flight delays, achieving a recall of **0.676** and an F1 score of **0.390**.

In **Phase 3**, we significantly expanded the scope of our analysis by scaling the dataset to encompass the entire period from 2015 to 2024. Additionally, we engineered domain-specific features that provided valuable insights into potential delay triggers, such as `origin_delay_rate`, `triplet`, `airport_congestion_level`, and `previous_leg_delayed`. We evaluated a range of advanced models, including **XGBoost**, **MLP (Multilayer Perceptron)**, and **BiLSTM**. The **best-performing model** emerged as a single-layer MLP with 448 hidden units, a dropout rate of 0.276, L2 regularization of 0.000155, a batch size of 256, and a learning rate of approximately **1e-5**. This model achieved remarkable performance, achieving an **F2 score of 0.746**, **Recall of 0.752**, and **Precision of 0.724** on the 2024 test set—strongly validating the effectiveness of this architecture and feature design.

As outlined in the Results and Discussion section, accurately identifying **three-quarters of delayed flights** presents a significant opportunity for substantial operational savings. Even a modest **5% reduction in delay-related costs** amounts to over **$11 million** in annual savings—and over **$140 million** over a 10-year period. These findings validate the operational viability and financial worth of this predictive system.

Moving forward, the next phase involves collaborating with stakeholders across Southwest’s operations to pilot a **soft launch** of this delay classifier. Crucially, all input features utilized by the model are accessible **at least 15 minutes prior to departure**, ensuring its real-time feasibility. While we can adjust this time buffer to a larger value, such as 30 minutes prior, to facilitate earlier identification, this may result in a slight performance compromise. Therefore, during the soft-launch phase, we aim to assess whether an at least 15-minute-early identification is sufficient. If successful, we can gradually expand its application to all Southwest flights, leading to measurable savings and enhancing the customer experience.

### **Meet the Team - Group 4_4**

<table>
  <tr>
    <td align="center">
      <img src="https://ca.slack-edge.com/T0WA5NWKG-U068N7K7D71-275409804ab1-512" width="100"><br>
      <strong>Ayushi Goel</strong><br>
      ✉️ <a href="mailto:ayugoel@ischool.berkeley.edu">Email</a>
    </td>
    <td align="center">
      <img src="https://ca.slack-edge.com/T0WA5NWKG-U050HPGAQ7Q-75fe6a983dc6-512" width="100"><br>
      <strong>Licheng Zhong</strong><br>
      ✉️ <a href="mailto:lzmids@ischool.berkeley.edu">Email</a>
    </td>
    <td align="center">
      <img src="https://ca.slack-edge.com/T0WA5NWKG-U061UNNDZPH-968279cfd6bb-512" width="100"><br>
      <strong>Louis Wu</strong><br>
      ✉️ <a href="mailto:louiswu1201@ischool.berkeley.edu">Email</a>
    </td>
    <td align="center">
      <img src="https://ca.slack-edge.com/T0WA5NWKG-U05E8ARFHAR-ad9dda100c72-512" width="100"><br>
      <strong>Sammy Cayo</strong><br>
      ✉️ <a href="mailto:sacayo@ischool.berkeley.edu">Email</a>
    </td>
  </tr>
</table>

#### [Code notebook](https://dbc-fae72cab-cf59.cloud.databricks.com/editor/notebooks/3336123436790193?o=4021782157704243)

#### Appendix: Clarification of Financial Calculations

The financial estimates provided in this project are **approximations** and should be interpreted with caution. The total economic impact of flight delays and potential savings from reducing delays are derived from **publicly available data**, **not Southwest Airlines' internal financial records**.

#### Source of Economic Impact Estimate:
The cost per delayed flight is taken from the article **["Flight Delays in Numbers – Not Only Painful For Passengers"](https://www.allthingsontimeperformance.com/flight-delays-in-numbers-not-only-painful-for-passengers/)**, which states that **each delayed flight costs approximately $920**.

#### Calculation Details:

- **Total Delayed Flights (2015–2021):** **3.09 million**  
- **Cost Per Delayed Flight:** **$920**  
- **Total Economic Impact of Delays:**  
  - **3,090,000 × 920 = $2.84 billion**  

- **Potential Savings from a 5% Reduction in Delays:**
  - **5% of Total Delayed Flights:**  
    - **3,090,000 × 0.05 = 154,500 flights**  
  - **Potential Savings:**  
    - **154,500 × 920 = $142.1 million** 

#### Model Prediction vs Actual Delays
- **2024 total delay: 340,115**
- **True prediction delay: 256,012**
- **Delay opportunity**
  - 256,012 (True Positives) * 920  ~ $235 Million
- **5% cost reduction**
  - 256,012 * 5% * 920 = 11,845.6 * 920 ~ $11.7 Million 
- **10% cost reduction**
  - 256,012 * 10% * 920 =  23,691 * 920 ~  $23 Million

<a name="appendix"></a>